<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Juan Aranda
- Nombre de alumno 2: Nicolás Crocco


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/Juanaranda/Aranda_Crocco`

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

path = '/content/drive/MyDrive/Laboratorios/Lab_09'
path1 = '/content/drive/MyDrive/12vo Semestre/Laboratorio de Programación Científica/Laboratorios/Lab_09'

df = pd.read_csv(path+'/sales.csv')
#df = pd.read_csv(path1+'/sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [4]:
# Otras librerías útiles
import datetime as dt

# Graficar
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import matplotlib.pyplot as plt

import xgboost as xgb

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

import joblib

## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [5]:
# EDA
print('Tipos de las columnas del dataset:')
print(df.dtypes)
print('-------------------')
print('Valores únicos por columna del dataset:')
print(df.nunique())
print('-------------------')

Tipos de las columnas del dataset:
id                    int64
date         datetime64[ns]
city                 object
lat                 float64
long                float64
pop                   int64
shop                 object
brand                object
container            object
capacity             object
price               float64
quantity              int64
dtype: object
-------------------
Valores únicos por columna del dataset:
id           7456
date           84
city            5
lat             6
long            6
pop            35
shop            6
brand           5
container       3
capacity        3
price         402
quantity     6906
dtype: int64
-------------------


Vemos la existencia de las columnas "lat", "long" y "city". Las primeras dos definen la ubicación en coordenadas polares de donde se realizó la venta, y la variable "city" muestra la ciudad donde está ubicada la tienda en donde se realizó la venta. Por lo anterior, eliminaremos las columnas "lat" y "long", manteniendo la columna "city" para obtener la ubicación de la venta. También se puede eliminar la columna "id", ya que no aporta información útil.
Por otro lado, debido a la poca cantidad de valores únicos, vemos que las siguientes columnas pueden tratarse como tipo categórico: {city, shop, brand, container, capacity}

In [6]:
# Eliminamos las columnas "lat" y "long"
df_prep = df.copy()
df_prep = df_prep.drop(['id', 'lat', 'long'], axis=1)
df_prep

,date,city,pop,shop,brand,container,capacity,price,quantity
0,2012-01-31,Athens,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,2012-01-31,Athens,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2012-01-31,Athens,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,2012-01-31,Athens,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,2012-01-31,Athens,672130,shop_1,adult-cola,can,330ml,0.39,25696
...,...,...,...,...,...,...,...,...,...
7451,2018-12-31,Athens,664046,shop_1,kinder-cola,plastic,1.5lt,2.52,13760
7452,2018-12-31,Athens,664046,shop_1,orange-power,plastic,1.5lt,2.18,16309
7453,2018-12-31,Patra,168034,shop_6,kinder-cola,can,330ml,0.85,24378
7454,2018-12-31,Thessaloniki,354290,shop_4,adult-cola,plastic,1.5lt,2.17,20691


In [7]:
# Transformamos las columnas definidas a categóricas
columnas_cat = ['city', 'shop', 'brand', 'container', 'capacity']
for col in columnas_cat:
  df_prep[col] = df_prep[col].astype('category')

print('Tipos de las columnas del dataset:')
print(df_prep.dtypes)


Tipos de las columnas del dataset:
date         datetime64[ns]
city               category
pop                   int64
shop               category
brand              category
container          category
capacity           category
price               float64
quantity              int64
dtype: object


In [8]:
# 1. Separar datos (train, validation y test)
seed = 7428491 # Fijar semilla

# Separar en features y target
y = df_prep['quantity']
X = df_prep.drop('quantity', axis=1)

# Separar en train, validation y test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=seed)

In [9]:
# 2. FunctionTransformer para extraer día, mes y año de variable 'date'
def getDate(X):
    df = X.copy()
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    df.drop('date', axis=1, inplace=True)  # Eliminar la columna 'date'
    return df

# Crear un objeto FunctionTransformer con la función getDate
date_transformer = FunctionTransformer(getDate)

# Probar el transformer
df_transformed = date_transformer.transform(X_train.copy())
df_transformed

,city,pop,shop,brand,container,capacity,price,day,month,year
7133,Thessaloniki,354290,shop_4,orange-power,plastic,1.5lt,2.09,30,9,2018
5698,Patra,168501,shop_6,orange-power,plastic,1.5lt,1.73,31,5,2017
4378,Patra,168254,shop_6,lemon-boost,plastic,1.5lt,2.49,29,2,2016
5590,Irakleion,138200,shop_2,kinder-cola,plastic,1.5lt,2.97,30,4,2017
1000,Irakleion,134219,shop_2,lemon-boost,plastic,1.5lt,2.13,31,12,2012
...,...,...,...,...,...,...,...,...,...,...
6571,Irakleion,137154,shop_2,adult-cola,can,330ml,0.69,31,3,2018
6845,Irakleion,137154,shop_2,orange-power,can,330ml,0.43,30,6,2018
1174,Irakleion,135432,shop_2,lemon-boost,glass,500ml,1.01,28,2,2013
3375,Athens,667237,shop_1,orange-power,glass,500ml,0.88,31,3,2015


In [10]:
# 3. ColumnTransformer para procesar variables numéricas y categóricas

numeric_features = df_transformed.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df_transformed.select_dtypes(include=['category']).columns.tolist()

#num_transformer = Pipeline(steps=('num', 'passthrough'))
cat_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformers = [
    ('cat', cat_transformer, categorical_features) # OneHotEncoder para todas las variables categóricas
    ]

preprocessor = ColumnTransformer(transformers, remainder='passthrough')

#preprocessor.set_output(transform="pandas")

# Aplicar el ColumnTransformer al DataFrame original
df_transformed = preprocessor.fit_transform(df_transformed)

# Obtener columnas del df transformado
column_names = preprocessor.get_feature_names_out()

# Transformar resultado a DataFrame
df_processed = pd.DataFrame(df_transformed.toarray(), columns=column_names)

# Mostrar el DataFrame resultante
df_processed.head()

,cat__city_Athens,cat__city_Irakleion,cat__city_Larisa,cat__city_Patra,cat__city_Thessaloniki,cat__shop_shop_1,cat__shop_shop_2,cat__shop_shop_3,cat__shop_shop_4,cat__shop_shop_5,...,cat__month_12,cat__year_2012,cat__year_2013,cat__year_2014,cat__year_2015,cat__year_2016,cat__year_2017,cat__year_2018,remainder__pop,remainder__price
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,354290.0,2.09
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,168501.0,1.73
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,168254.0,2.49
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,138200.0,2.97
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,134219.0,2.13


In [11]:
# 4. DummyRegressor pipeline
from sklearn.dummy import DummyRegressor

# Pipeline con preprocesamiento y DummyRegressor como regresor
pipeline_dummy = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor())
])

# Entrenar el pipeline con DummyRegressor
pipeline_dummy.fit(X_train, y_train)

predictions_dummy = pipeline_dummy.predict(X_val)

mae_dummy = mean_absolute_error(y_val, predictions_dummy)

print(f"MAE con DummyRegressor: {mae_dummy}")

MAE con DummyRegressor: 13332.920828541537


In [12]:
# 5. Pipeline con preprocesamiento y XGBRegressor como regresor
# Separar en train, validation y test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=seed)

pipeline_xgb = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor())  # Usando XGBRegressor
])

# Entrenar el pipeline con XGBRegressor
pipeline_xgb.fit(X_train, y_train)
predictions_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, predictions_xgb)
print(f"MAE con XGBRegressor: {mae_xgb}")

MAE con XGBRegressor: 2529.0266605328584


In [13]:
# 6. Determinar si XGBRegressor es mejor o peor que DummyRegressor
if mae_xgb < mae_dummy:
    print("El modelo XGBRegressor es mejor que el DummyRegressor.")
elif mae_xgb > mae_dummy:
    print("El modelo XGBRegressor es peor que el DummyRegressor.")
else:
    print("Ambos modelos tienen un desempeño similar.")

El modelo XGBRegressor es mejor que el DummyRegressor.


In [14]:
# 7. Guardar los modelos en archivos .pkl
joblib.dump(pipeline_dummy, 'dummy_regressor_model.pkl')
joblib.dump(pipeline_xgb, 'xgb_regressor_model.pkl')

['xgb_regressor_model.pkl']

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

### Ocupando el pipeline

In [19]:
# Separar en train, validation y test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=seed)

# Entrena el pipeline sin restricciones
pipeline_xgb.fit(X_train, y_train)

# Realiza predicciones en el conjunto de validación sin restricciones
predictions_no_constraints = pipeline_xgb.predict(X_val)

# Calcula el MAE sin restricciones
mae_no_constraints = mean_absolute_error(y_val, predictions_no_constraints)
print(f"MAE sin restricciones monotónicas: {mae_no_constraints}")

# Pipeline con preprocesamiento y XGBRegressor con restricción monotónica
pipeline_xgb_constrained = Pipeline(steps=[
    ('date_transformer', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor())  # Usando XGBRegressor
])
#monotone_constraints={'remainder__price': -1}
monotone_constraints = tuple([0] * 46 + [-1])
# Aplicar restricciones monotónicas directamente en el XGBRegressor
xgb_regressor = pipeline_xgb_constrained.named_steps['regressor']
xgb_regressor.set_params(monotone_constraints=monotone_constraints)

# Entrenar el pipeline con restricciones
pipeline_xgb_constrained.fit(X_train, y_train)


# Realizar predicciones en el conjunto de validación
predictions_constrained = pipeline_xgb_constrained.predict(X_val)

# Calcular el MAE con restricciones
mae_constrained = mean_absolute_error(y_val, predictions_constrained)
print(f"MAE con restricciones monotónicas: {mae_constrained}")

# Guarda el modelo con restricciones en un archivo .pkl
joblib.dump(pipeline_xgb_constrained, 'pipeline_xgb_constrained.pkl')

MAE sin restricciones monotónicas: 2529.0266605328584
MAE con restricciones monotónicas: 2473.050808410849


['pipeline_xgb_constrained.pkl']

El amigo tenía razón puesto que se tiene un MAE menor con restricciones monotónicas (2473.05) en comparación con el MAE sin restricciones (2529.03). De acuerdo a los resultados anteriores, se puede asumir que forzar una relación monótona negativa entre el precio y la cantidad ha ayudado a mejorar el rendimiento del modelo. Lo que sugiere, que efectivamente había una relación inversa entre el precio y la cantidad en los datos, lo cual ha sido capturado por el modelo al imponer esta restricción.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

**Explicación de cada hiperparámetro y su rol en el modelo**

    learning_rate:
        Tasa de aprendizaje que controla la contribución de cada árbol al modelo y afecta la velocidad de convergencia.
        El rango de [0.001, 0.1] es razonable.

    n_estimators:
        Número de árboles que generará el modelo.
        El rango de [50, 1000] es razonable, sin embargo, el límite superior podría ser muy alto, por lo que podría  ser ajustado dependiendo del tiempo y recursos computacionales disponibles.

    max_depth:
        Máxima profundidad de cada árbol. Controla la complejidad del modelo y su capacidad de adaaptación frente a patrones en los datos.
        El rango de [3, 10] es razonable.

    max_leaves:
        Máximo número de nodos terminales, u hojas, en cada árbol.
        El rango de [0, 100] es razonable y permite poner un límite a la cantidad de hojas en árboles con las profundidades definidas en el parámetro anterior.

    min_child_weight:
        Controla la cantidad mínima de instancias necesarias en un nodo hoja.
        El rango de [1, 5] es razonable.

    reg_alpha:
        Término de regularización L1 en los pesos de las hojas.
        El rango de [0, 1] es razonable.

    reg_lambda:
        Término de regularización L2 en los pesos de las hojas.
        Rango sugerido: El rango de [0, 1] es razonable.

In [16]:
!pip install -U optuna

In [17]:
import optuna
from sklearn.model_selection import cross_val_score

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=1/3, random_state=seed)

# Define la función objetivo
def objective(trial):
    # Define la estructura de tu pipeline con los parámetros que quieres optimizar
    #preprocessor = ColumnTransformer(transformers, remainder='passthrough')

    # Definir el modelo XGBRegressor dentro del pipeline
    pipeline_xgb = Pipeline(steps=[
        ('date_transformer', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(
            learning_rate=trial.suggest_float('learning_rate', 0.001, 0.1),
            n_estimators=trial.suggest_int('n_estimators', 50, 1000),
            max_depth=trial.suggest_int('max_depth', 3, 10),
            max_leaves=trial.suggest_int('max_leaves', 0, 100),
            min_child_weight=trial.suggest_int('min_child_weight', 1, 5),
            reg_alpha=trial.suggest_float('reg_alpha', 0, 1),
            reg_lambda=trial.suggest_float('reg_lambda', 0, 1),
            seed=seed
        ))
    ])

    # Entrena y evalúa el modelo con validación cruzada
    scores = -cross_val_score(pipeline_xgb, X_train, y_train, cv=5, scoring='neg_mean_absolute_error', error_score='raise')

    # Almacena el mejor modelo en user_attrs
    pipeline_xgb.fit(X_train, y_train)
    trial.set_user_attr('pipeline', pipeline_xgb)

    return scores.mean()  # Retornamos el promedio del MAE negativo

# Configurar el estudio de Optuna
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(sampler=sampler, direction='minimize')

# Ejecutar la optimización durante 5 minutos
study.optimize(objective, n_trials=100, timeout=300)

# Obtener los mejores parámetros y el MAE correspondiente
best_params = study.best_params
best_mae = study.best_value

print(f"Number of trials: {len(study.trials)}")
print(f"Best MAE: {best_mae}")
print(f"Best parameters: {best_params}")

# Guardar el mejor modelo en un archivo .pkl
best_pipeline = study.best_trial.user_attrs['pipeline']
joblib.dump(best_pipeline, 'best_xgb_model.pkl')

[I 2023-11-17 16:23:57,605] A new study created in memory with name: no-name-9c9de819-a815-488e-ad16-d04c6df36c6d
[I 2023-11-17 16:24:06,758] Trial 0 finished with value: 2484.423966266606 and parameters: {'learning_rate': 0.03644185849101597, 'n_estimators': 358, 'max_depth': 7, 'max_leaves': 81, 'min_child_weight': 3, 'reg_alpha': 0.7579693455411626, 'reg_lambda': 0.4826414427681184}. Best is trial 0 with value: 2484.423966266606.
[I 2023-11-17 16:24:11,434] Trial 1 finished with value: 3904.6933094808455 and parameters: {'learning_rate': 0.00660614718129643, 'n_estimators': 737, 'max_depth': 5, 'max_leaves': 74, 'min_child_weight': 2, 'reg_alpha': 0.7352457603786917, 'reg_lambda': 0.7368771651349628}. Best is trial 0 with value: 2484.423966266606.
[I 2023-11-17 16:24:25,388] Trial 2 finished with value: 2328.031412023662 and parameters: {'learning_rate': 0.03863884750695508, 'n_estimators': 782, 'max_depth': 7, 'max_leaves': 82, 'min_child_weight': 3, 'reg_alpha': 0.569483096976287,

Number of trials: 34
Best MAE: 2244.3500856510527
Best parameters: {'learning_rate': 0.09980145372956829, 'n_estimators': 805, 'max_depth': 6, 'max_leaves': 52, 'min_child_weight': 2, 'reg_alpha': 0.23554839421930945, 'reg_lambda': 0.62125174358005}


['best_xgb_model.pkl']

Podemos notar que el mejor modelo obtenido de la optimización de parámetros mediante optuna reporta un mejor MAE que en los casos anteriores. Esto puede deberse, en primer lugar, al abanico de posibilidades de hiperparámetros que se le entregó al optimizador, lo que le permitió hacer muchas más pruebas que en los casos anteriores. Además, se le dio un tiempo máximo para la optimización de 5 minutos, lo que se complementa con la cantidad de combinaciones, permitiéndole hacer más de 35 pruebas con distintos hiperparámetros.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?**

El "pruning", o poda en español, se refiere a la técnica de detener aniticipadamente el entrenamiento de un modelo si se observa que su rendimiento no mejora significativamente. Esto se logra evaluando regularmente el modelo en un conjunto de validación durante el entrenamiento. Antes de completar todas las iteraciones especificadas, se detiene el entrenamiento si el rendimiento no mejora lo suficiente.

El "pruning" impacta en el entrenamiento al ayudar a evitar el sobreajuste y reducir la cantidad de tiempo que se dedica al entrenamiento, por lo que se esperaría que se cree un modelo más efectivo y generalizable al detenerse el entrenamiento cuando no se encuentren mejoras relevantes.

In [21]:
from optuna.integration import XGBoostPruningCallback

# Configurar el estudio de Optuna con prunning
study_with_pruning = optuna.create_study(sampler=sampler, direction='minimize', pruner=XGBoostPruningCallback(trial=optuna.trial.Trial, observation_key='validation-error'))

# Ejecutar la optimización con prunning durante 5 minutos
study_with_pruning.optimize(objective, n_trials=100, timeout=300)

# Obtener los mejores parámetros y el MAE correspondiente con prunning
best_params_with_pruning = study_with_pruning.best_params
best_mae_with_pruning = study_with_pruning.best_value

# Resultados sin prunning
print(f"Number of trials without pruning: {len(study.trials)}")
print(f"Best MAE without pruning: {best_mae}")
print(f"Best parameters without pruning: {best_params}")

# Resultados con prunning
print(f"Number of trials with pruning: {len(study_with_pruning.trials)}")
print(f"Best MAE with pruning: {best_mae_with_pruning}")
print(f"Best parameters with pruning: {best_params_with_pruning}")

# Guardar el mejor modelo con prunning en un archivo .pkl
best_pipeline_with_pruning = study_with_pruning.best_trial.user_attrs['pipeline']
joblib.dump(best_pipeline_with_pruning, 'best_xgb_model_with_pruning.pkl')


[I 2023-11-17 16:59:48,694] A new study created in memory with name: no-name-8ea23681-f6df-43f2-8aa5-56c08d01cf5b
[I 2023-11-17 16:59:57,051] Trial 0 finished with value: 10502.972610634897 and parameters: {'learning_rate': 0.009225839045546792, 'n_estimators': 75, 'max_depth': 10, 'max_leaves': 10, 'min_child_weight': 5, 'reg_alpha': 0.29256320030891125, 'reg_lambda': 0.006888250146942165}. Best is trial 0 with value: 10502.972610634897.
[I 2023-11-17 17:00:02,902] Trial 1 finished with value: 2330.058472399761 and parameters: {'learning_rate': 0.08479284646074982, 'n_estimators': 420, 'max_depth': 7, 'max_leaves': 88, 'min_child_weight': 2, 'reg_alpha': 0.4108496400402669, 'reg_lambda': 0.638135058697166}. Best is trial 1 with value: 2330.058472399761.
[I 2023-11-17 17:00:13,421] Trial 2 finished with value: 2316.5095667221785 and parameters: {'learning_rate': 0.09652201886871664, 'n_estimators': 505, 'max_depth': 6, 'max_leaves': 98, 'min_child_weight': 1, 'reg_alpha': 0.53909582998

Number of trials without pruning: 34
Best MAE without pruning: 2244.3500856510527
Best parameters without pruning: {'learning_rate': 0.09980145372956829, 'n_estimators': 805, 'max_depth': 6, 'max_leaves': 52, 'min_child_weight': 2, 'reg_alpha': 0.23554839421930945, 'reg_lambda': 0.62125174358005}
Number of trials with pruning: 36
Best MAE with pruning: 2242.3592510426906
Best parameters with pruning: {'learning_rate': 0.08253296325875228, 'n_estimators': 767, 'max_depth': 9, 'max_leaves': 43, 'min_child_weight': 4, 'reg_alpha': 0.7666724762025932, 'reg_lambda': 0.7926501663270636}


['best_xgb_model_with_pruning.pkl']

Se puede observar que el MAE obtenido mediante la pruning es ligeramente menor que el MAE obtenido sin la poda.

Esta mejora podría deberse a que el pruning ayuda a evitar que el modelo continúe en direcciones que no conducen a mejoras significativas en la métrica objetivo. Al interrumpir tempranamente las combinaciones de hiperparámetros que no muestran mejoras sustanciales, Optuna puede explorar de manera más eficiente el espacio de búsqueda y enfocarse en las configuraciones que son más prometedoras.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [22]:
!pip install optuna-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 10.1 MB/s eta 0:00:00


In [23]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

# Visualizar el historial de optimización
plot_optimization_history(study)

Analizando el gráfico del historial de optimización, se puede apreciar que a partir del 9no Trial se empiezan a ver mejoras notables en los resultados. Es importante mencionar que se llegó al mejor resultado en el Trial 14.

In [24]:
# Visualizar el gráfico de coordenadas paralelas
plot_parallel_coordinate(study)

En primer lugar, se puede ver una gran diversidad de caminos tomados por la optimización de parámetros, lo que indica que hubo una buena exploración durante el estudio. <br>
Se puede ver una clara tendencia a buscar el siguiente "camino": bajo reg_alpha, alto número de n_estimators, bajo min_child_weight, alto max_leaves, max_depth presenta mayor diversidad de caminos por lo que no tiene una tendencia clara, y finalmente, un alto learning rate. <br>
De todas maneras, al analizar el el "camino" del mejor Trial, es posible notar que toma valores relativamente medios en todos los hiperparámentros, a excepción del learning rate, donde llega al máximo valor que se probó en la optimización de hiperparámetros¡.

In [27]:
# Visualizar el gráfico de importancia de hiperparámetros
plot_param_importances(study)

Es claro ver que los tres hiperparámetros más relevantes son max_leaves, learning_rate y n_estimators. Donde max_leaves es por lejos el parámetro más importante. Sobre esto, aunque no es posible verlo con tanta claridad, se puede obtener vagamente del gráfico de coordenadas paralelas que los tres Trials que se realizaron con max_leaves=50 (aproximadamente), obtuvieron resultados objetivos similares y cercanos al mejor valor objetivo.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [33]:
# MAE de cada modelo

mae_dict = {
    'Modelo': [
        'Baseline Dummy',
        'Baseline XGBoost',
        'XGBoost con Constraints',
        'XGBoost con Optuna',
        'XGBoost con Optuna y Prunning'
    ],
    'MAE': [
        mae_dummy,
        mae_xgb,
        mae_constrained,
        best_mae,
        best_mae_with_pruning
    ]
}

# Crear un DataFrame a partir del diccionario
mae_df = pd.DataFrame(mae_dict)

# Mostrar la tabla resumen de MAE
print(mae_df)

                          Modelo           MAE
0                 Baseline Dummy  13332.920829
1               Baseline XGBoost   2529.026661
2        XGBoost con Constraints   2473.050808
3             XGBoost con Optuna   2244.350086
4  XGBoost con Optuna y Prunning   2242.359251


Según la tabla resumen, el modelo que muestra el mejor rendimiento basado en la métrica de error absoluto medio (MAE) es el modelo XGBoost con Optuna y Prunning. Este modelo logra minimizar el error de predicción de manera más efectiva en comparación con los otros modelos evaluados, reflejando una mejor capacidad para generalizar y predecir con precisión las cantidades en los datos de validación.

In [29]:
# Cargar el mejor modelo entrenado con XGBoost con Optuna y Pruning
best_model = joblib.load('best_xgb_model_with_pruning.pkl')

# Predicción en el conjunto de test
predictions_test = best_model.predict(X_test)
mae_test = mean_absolute_error(y_test, predictions_test)

print(f"MAE en el conjunto de test: {mae_test}")

MAE en el conjunto de test: 2235.327535920744


Al evaluar el modelo XGBoost con Optuna y Prunning en el conjunto de pruebas, se nota una leve mejora en su rendimiento en comparación con los resultados obtenidos en el conjunto de validación, dado que el MAE disminuye ligeramente. Esto puede deberse a la capacidad del modelo para generalizar patrones con mayor eficacia en datos no vistos, lo que resulta en una mejor precisión en las predicciones. Esta mejora podría atribuirse a la capacidad del modelo para ajustarse de manera más óptima a la variabilidad natural y a la distribución de los datos de prueba, permitiendo una mejor adaptación a las particularidades presentes en este conjunto de datos específico.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>